In [30]:
import os
import shutil

# 设置路径
data_dir = './videos/MRI515_T1'
# 
label_dir = os.path.join(data_dir, 'data_in_jpg_raw_name/label_in_png_T1_DINO_order')
# label_dir = os.path.join(data_dir, 'data_in_jpg_raw_name/label_in_png_T2_DINO_order')


prompt_dir = 'manual_prompt_frame0_1_2'


auto_seg_dir = os.path.join(data_dir, prompt_dir, 'SAM2_seg_mask_nolap')


# 创建新的目标目录
output_dir = os.path.join(data_dir, prompt_dir, 'seg_mask_renamed')
os.makedirs(output_dir, exist_ok=True)

# 获取 label_dir 中的文件名映射
label_mapping = {}
for label_file in os.listdir(label_dir):
    if label_file.endswith('.png'):
        file_prefix = label_file.split('_')[0]  # 获取开头的00000, 00001等
        label_mapping[file_prefix] = label_file

# 复制并重命名文件
for file in os.listdir(auto_seg_dir):
    if 'combined.png' in file:
        # 提取 frame_xx 编号
        frame_idx = file.split('_')[1]  # 获取 frame_xx 中的 xx
        frame_prefix = frame_idx.zfill(5)  # 确保前导零一致 (00000, 00001)

        # 查找匹配的 label 文件名
        if frame_prefix in label_mapping:
            new_filename = label_mapping[frame_prefix]
            src_path = os.path.join(auto_seg_dir, file)
            dst_path = os.path.join(output_dir, new_filename)
            
            if not os.path.exists(dst_path):
            # 复制并重命名文件
                shutil.copy(src_path, dst_path)
                print(f"Copied {file} → {new_filename}")

print(f"All files copied to {output_dir}")


All files copied to ./videos/MRI515_T1/manual_prompt_frame0_1_2/seg_mask_renamed


In [32]:
import os, glob,cv2
import SimpleITK as sitk
import numpy as np
from PIL import Image  # 用于读取 PNG 掩码

# -------------- 保存 NIfTI 文件的函数 --------------
def save_array_to_nii(mask_array, nii_template_path, nii_save_path):
    """
    将 NumPy 掩码数组保存为 NIfTI 格式，保持与模板 NIfTI 文件相同的空间信息。
    """
    sitk_image_object = sitk.ReadImage(nii_template_path)
    output_spacing = sitk_image_object.GetSpacing()
    output_direction = sitk_image_object.GetDirection()
    output_origin = sitk_image_object.GetOrigin()

    # 将 NumPy 数组转换为 SimpleITK 图像对象
    nii_output = sitk.GetImageFromArray(mask_array)
    nii_output.SetSpacing(output_spacing)
    nii_output.SetDirection(output_direction)
    nii_output.SetOrigin(output_origin)

    # 保存 NIfTI 文件
    sitk.WriteImage(nii_output, nii_save_path)
    print(f"Saved: {nii_save_path}")

# -------------- 目录路径 --------------
label_nii_dir = os.path.join(data_dir, 'data_in_jpg_raw_name/label_in_nii')  # 原始 NIfTI 文件
seg_nii_dir = os.path.join(data_dir,  prompt_dir, 'seg_in_nii')  # 保存分割 NIfTI 的目录
output_dir = os.path.join(data_dir, prompt_dir, 'seg_mask_renamed')  # 自动分割 PNG 掩码所在目录

# seg_nii_dir = os.path.join(data_dir, 'label_in_nii_72_L4-L5')  # 保存分割 NIfTI 的目录
# output_dir = os.path.join(data_dir, 'data_in_nii','label_in_png_L4L5_4class_KMeans_sorted')  # 自动分割 PNG 掩码所在目录


os.makedirs(seg_nii_dir, exist_ok=True)  # 创建保存路径（如果不存在）

# -------------- 遍历自动分割的掩码文件 --------------
for idx, filename in enumerate(os.listdir(output_dir)):    
    if '0132_T1' in filename or '0356_T2' in filename or '0052_T1' in filename:
        print(f"{idx}: Processing {filename}")

        # 拆分文件名，提取 `pid` 和 `slice_idx`
        parts = filename.split('_')
        pid = "_".join(parts[1:-4])  # 提取 PID，例如 "ID1A_R0"
        slice_idx = int(parts[-3].replace('slice', ''))  # 提取 Slice Index，例如 017 → 17

        # 查找匹配的 NIfTI 文件
        nii_files = glob.glob(os.path.join(label_nii_dir, pid + "*.nii.gz"))
        if not nii_files:
            print(f"Warning: No matching NIfTI file found for PID {pid}")
            continue
        label_nii_path = nii_files[0]

        # 读取 label NIfTI 作为模板
        label_sitk = sitk.ReadImage(label_nii_path)
        label_array = sitk.GetArrayFromImage(label_sitk)  # 转换为 NumPy 数组

        # 创建新的 mask_array（与 label_array 形状相同，全为 0）
        mask_array = np.zeros_like(label_array, dtype=np.uint8)

        # 读取 PNG 掩码文件（auto_mask）
        mask_path = os.path.join(output_dir, filename)
        auto_mask = np.array(Image.open(mask_path))

       # 获取 mask_array[slice_idx, :, :] 的目标形状
        target_shape = mask_array[slice_idx, :, :].shape  # (H, W)

        # 初始化新的 mask_mapped_resized 数组（与目标形状相同）
        mask_mapped_resized = np.zeros(target_shape, dtype=np.uint8)

        # 逐个处理每个 class，进行 resize 并合并
        for class_value in [50, 100, 150, 200]:
            binary_mask = (auto_mask == class_value).astype(np.uint8)  # 生成二值 mask
            resized_binary_mask = cv2.resize(binary_mask, (target_shape[1], target_shape[0]), interpolation=cv2.INTER_NEAREST)  # 保持最近邻插值
            mask_mapped_resized[resized_binary_mask > 0] = {50: 1, 100: 2, 150: 3, 200: 4}[class_value]  # 赋值回正确的 class

        # 赋值到 mask_array
        if 'ID' in pid:
            mask_mapped_resized =np.rot90(mask_mapped_resized, 2) 
        mask_array[slice_idx, :, :] = mask_mapped_resized

        # 设置保存路径
        nii_save_path = os.path.join(seg_nii_dir, f"{pid}_L4-L5_sam2_seg_slice{slice_idx}_seg.nii.gz")

        # 保存 NIfTI 掩码文件
        save_array_to_nii(mask_array, label_nii_path, nii_save_path)

        print(f"PID: {pid}, Slice Index: {slice_idx}")
        print(f"Label NIfTI Path: {label_nii_path}")
        print(f"Saved Segmentation: {nii_save_path}\n\n")


88: Processing 00044_0132_T1_slice_006_imgseg_.png
Saved: ./videos/MRI515_T1/manual_prompt_frame0_1_2/seg_in_nii/0132_T1_L4-L5_sam2_seg_slice6_seg.nii.gz
PID: 0132_T1, Slice Index: 6
Label NIfTI Path: ./videos/MRI515_T1/data_in_jpg_raw_name/label_in_nii/0132_T1_TSE_TRA_img_nnunet_to_seg_expand-segmentation-label.nii.gz
Saved Segmentation: ./videos/MRI515_T1/manual_prompt_frame0_1_2/seg_in_nii/0132_T1_L4-L5_sam2_seg_slice6_seg.nii.gz


137: Processing 00058_0052_T1_slice_006_imgseg_.png
Saved: ./videos/MRI515_T1/manual_prompt_frame0_1_2/seg_in_nii/0052_T1_L4-L5_sam2_seg_slice6_seg.nii.gz
PID: 0052_T1, Slice Index: 6
Label NIfTI Path: ./videos/MRI515_T1/data_in_jpg_raw_name/label_in_nii/0052_T1_TSE_TRA_img_nnunet_to_seg_expand-segmentation-label.nii.gz
Saved Segmentation: ./videos/MRI515_T1/manual_prompt_frame0_1_2/seg_in_nii/0052_T1_L4-L5_sam2_seg_slice6_seg.nii.gz




In [29]:
# 获取两个文件夹中的文件名并提取数字 ID（去掉 _T1 或 _T2）
folder_path_T1 = './videos/MRI515_T1/data_in_jpg_raw_name/img_in_png_SAM_L4-L5_4class_T1'
folder_path_T2 = './videos/MRI515_T2/data_in_jpg_raw_name/img_in_png_SAM_L4-L5_4class_T2'

# 提取 T1 文件夹中的 ID
file_ids_T1 = set()
if os.path.exists(folder_path_T1):
    for filename in os.listdir(folder_path_T1):
        if filename.endswith(".jpg"):
            file_id = filename.split("_")[0]  # 提取数字 ID
            file_ids_T1.add(file_id)

# 提取 T2 文件夹中的 ID
file_ids_T2 = set()
if os.path.exists(folder_path_T2):
    for filename in os.listdir(folder_path_T2):
        if filename.endswith(".jpg"):
            file_id = filename.split("_")[0]  # 提取数字 ID
            file_ids_T2.add(file_id)

# 计算所有唯一的数字 ID（T1 和 T2 的并集）
all_unique_ids = file_ids_T1 | file_ids_T2
all_unique_ids = set(all_unique_ids)
all_unique_ids_count = len(all_unique_ids)
# 显示结果
common_ids_count
# 显示结果
print(len(file_ids_T1), len(file_ids_T2))
print(len(all_unique_ids))



415 411
417


In [25]:
len(file_ids_T2)

411

In [19]:
import os
import pandas as pd

# 假设文件夹路径
data_dir = './videos/MRI515_T1'
# 
folder_path = os.path.join(data_dir, 'data_in_jpg_raw_name/img_in_png_SAM_L4-L5_4class_T1')

# 获取文件夹中的文件名并提取 ID（形如 0001_T1）
file_ids = set()
for filename in os.listdir(folder_path):
    if filename.endswith(".jpg"):
        file_id = "_".join(filename.split("_")[:2])
        file_ids.add(file_id)

# 读取 CSV 文件
csv_path =os.path.join(data_dir, 'data_in_jpg_raw_name/MRI_Data_515_T1T2_nii_size.csv')
df = pd.read_csv(csv_path)  # 读取以制表符分隔的 CSV

print('len(file_ids) len:', len(file_ids))
# 获取 CSV 中的 ID
csv_ids = set(df[df["pid"].str.contains("T1")]["pid"])

# 找出 CSV 中存在但文件夹中不存在的 ID
extra_ids_in_csv1 = csv_ids - file_ids


# 提取对应的行，并选择 shape0 和 spacing0
extra_ids_details = df[df["pid"].isin(extra_ids_in_csv1)][["pid", "shape0", "spacing0"]]

extra_ids_details

len(file_ids) len: 415


pid  shape0  spacing0
0     0233_T1      48    0.6875
2     0027_T1      40    0.6875
5     0246_T1      40    0.6875
7     0243_T1      38    0.6875
8     0198_T1      32    0.6875
...       ...     ...       ...
480   0131_T1      16    0.6875
483   0395_T1      16    0.6875
485   0098_T1      16    0.6875
486   0139_T1      16    0.6875
1015  0505_T1      12    0.6875

[100 rows x 3 columns]

In [20]:
# 读取 CSV 文件
csv_path = os.path.join(data_dir, 'data_in_jpg_raw_name/MRI_Data_515_T1T2_nii_size.csv')
df = pd.read_csv(csv_path)

# 过滤出满足条件的行
condition = (df["shape0"].isin([20, 15, 12])) & (df["spacing0"].between(0.6865, 0.6885))
filtered_df = df[condition]

# 计算 T1 和 T2 的数量
num_T1 = filtered_df[filtered_df["pid"].str.contains("T1")].shape[0]
num_T2 = filtered_df[filtered_df["pid"].str.contains("T2")].shape[0]

# 计算唯一数字 ID 数量（去掉 _T1 和 _T2）
unique_ids = set(filtered_df["pid"].str[:-3])
num_unique_ids = len(unique_ids)

# 显示结果
num_T1, num_T2, num_unique_ids


(419, 414, 419)

In [3]:
# 读取 CSV 文件并仅保留 pid 中包含 "T1" 的行
df = pd.read_csv(csv_path)
df = df[df["pid"].str.contains("T1")]

# 过滤出满足 shape0 == 20, 15, 12 且 spacing0 约等于 0.6875 (误差小于 0.001) 的行
condition = (df["shape0"].isin([20, 15, 12])) & (df["spacing0"].between(0.6865, 0.6885))

# 提取这些行
filtered_df = df[condition]

# 找出 extra_ids_details 和 filtered_df 的差异
# 获取 extra_ids_details 和 filtered_df 中的 ID 集合
extra_ids_set = set(extra_ids_details["pid"])
filtered_ids_set = set(filtered_df["pid"])

# 找出 extra_ids_details 里有但 filtered_df 里没有的 ID
only_in_extra_ids = extra_ids_set - filtered_ids_set

# 找出 filtered_df 里有但 extra_ids_details 里没有的 ID
only_in_filtered = filtered_ids_set - extra_ids_set




In [10]:
len(extra_ids_set)

100

In [8]:
filtered_df.shape

(419, 8)

In [11]:
# 找出文件夹中缺失但 filtered_df 中存在的 ID
missing_ids_in_folder = set(filtered_df["pid"]) - file_ids

# 显示结果
missing_ids_in_folder


{'0156_T1', '0230_T1', '0447_T1', '0505_T1'}

In [29]:
# 过滤出满足 shape0 == 20, 15, 12 且 spacing0 约等于 0.6875 (误差小于 0.001) 的行
condition = (df["shape0"].isin([20, 15, 12])) & (df["spacing0"].between(0.6865, 0.6885))

# 提取这些行
filtered_df = df[condition]

# 检查 extra_ids_details 是否等于 filtered_df
is_identical = extra_ids_details.equals(filtered_df)

# 显示结果
is_identical


False

In [14]:
import os
import pandas as pd

# 假设文件夹路径
data_dir = './videos/MRI515_T2'
# 
folder_path = os.path.join(data_dir, 'data_in_jpg_raw_name/img_in_png_SAM_L4-L5_4class_T2')

# 获取文件夹中的文件名并提取 ID（形如 0001_T1）
file_ids = set()
for filename in os.listdir(folder_path):
    if filename.endswith(".jpg"):
        file_id = "_".join(filename.split("_")[:2])
        file_ids.add(file_id)

# 读取 CSV 文件
csv_path =os.path.join(data_dir, 'data_in_jpg_raw_name/MRI_Data_515_T1T2_nii_size.csv')
df = pd.read_csv(csv_path)  # 读取以制表符分隔的 CSV

print('len(file_ids) len:', len(file_ids))
# 获取 CSV 中的 ID
csv_ids = set(df[df["pid"].str.contains("T2")]["pid"])

# 找出 CSV 中存在但文件夹中不存在的 ID
extra_ids_in_csv2 = csv_ids - file_ids

# 显示结果
extra_ids_in_csv2
print('extra_ids len:', len(extra_ids_in_csv2))


len(file_ids) len: 411
extra_ids len: 104


In [17]:
# 读取 CSV 文件并仅保留 pid 中包含 "T1" 的行
df = pd.read_csv(csv_path)
df = df[df["pid"].str.contains("T2")]

# 过滤出满足 shape0 == 20, 15, 12 且 spacing0 约等于 0.6875 (误差小于 0.001) 的行
condition = (df["shape0"].isin([20, 15, 12])) & (df["spacing0"].between(0.6865, 0.6885))

# 提取这些行
filtered_df = df[condition]
print(filtered_df.shape)
# 找出文件夹中缺失但 filtered_df 中存在的 ID
missing_ids_in_folder = set(filtered_df["pid"]) - file_ids

# 显示结果
missing_ids_in_folder


(414, 8)


{'0124_T2', '0230_T2', '0447_T2'}

In [37]:
# 找出 extra_ids_details 和 filtered_df 的差异
# 获取 extra_ids_details 和 filtered_df 中的 ID 集合
extra_ids_set = set(extra_ids_details["pid"])
filtered_ids_set = set(filtered_df["pid"])

# 找出 extra_ids_details 里有但 filtered_df 里没有的 ID
only_in_extra_ids = extra_ids_set - filtered_ids_set

# 找出 filtered_df 里有但 extra_ids_details 里没有的 ID
only_in_filtered = filtered_ids_set - extra_ids_set

# 显示结果
only_in_extra_ids, only_in_filtered


({'0011_T1',
  '0020_T1',
  '0027_T1',
  '0029_T1',
  '0030_T1',
  '0035_T1',
  '0036_T1',
  '0037_T1',
  '0042_T1',
  '0055_T1',
  '0065_T1',
  '0070_T1',
  '0073_T1',
  '0075_T1',
  '0077_T1',
  '0082_T1',
  '0091_T1',
  '0098_T1',
  '0099_T1',
  '0104_T1',
  '0107_T1',
  '0114_T1',
  '0118_T1',
  '0119_T1',
  '0131_T1',
  '0133_T1',
  '0139_T1',
  '0144_T1',
  '0146_T1',
  '0156_T1',
  '0159_T1',
  '0160_T1',
  '0161_T1',
  '0163_T1',
  '0168_T1',
  '0180_T1',
  '0187_T1',
  '0191_T1',
  '0192_T1',
  '0198_T1',
  '0199_T1',
  '0200_T1',
  '0212_T1',
  '0230_T1',
  '0233_T1',
  '0243_T1',
  '0246_T1',
  '0250_T1',
  '0252_T1',
  '0263_T1',
  '0276_T1',
  '0281_T1',
  '0286_T1',
  '0290_T1',
  '0293_T1',
  '0301_T1',
  '0303_T1',
  '0305_T1',
  '0310_T1',
  '0317_T1',
  '0322_T1',
  '0324_T1',
  '0334_T1',
  '0337_T1',
  '0344_T1',
  '0350_T1',
  '0363_T1',
  '0366_T1',
  '0382_T1',
  '0385_T1',
  '0389_T1',
  '0395_T1',
  '0397_T1',
  '0399_T1',
  '0413_T1',
  '0415_T1',
  '0421_T1',

In [26]:
# 提取共同的 ID（去掉 T1/T2 后的数字）
common_ids = {id[:-3] for id in extra_ids_in_csv1} & {id[:-3] for id in extra_ids_in_csv2}

# 找出 extra_ids_in_csv1 里有但 extra_ids_in_csv2 里没有的 ID
only_in_extra_ids1 = {id[:-3] for id in extra_ids_in_csv1} - {id[:-3] for id in extra_ids_in_csv2}

# 找出 extra_ids_in_csv2 里有但 extra_ids_in_csv1 里没有的 ID
only_in_extra_ids2 = {id[:-3] for id in extra_ids_in_csv2} - {id[:-3] for id in extra_ids_in_csv1}

# 显示结果
only_in_extra_ids1


{'0156', '0505'}

In [27]:
only_in_extra_ids2

{'0124', '0272', '0368', '0403', '0445', '0515'}